In [ ]:
pip install scgraph>=2.13.0
pip install scgraph_data>=2.0.0
pip install folium==0.16.0

In [ ]:
from scgraph.geographs.marnet import marnet_geograph
from scgraph_data.world_highways import world_highways_geograph
from scgraph_data.world_railways import world_railways_geograph

# Get the shortest path between Marseille, France and Venice, Italy using Marnet, world highways, and world railways geographs
# origin = {"latitude": 43.2965,"longitude": 5.3698}  # Marseille, France
# destination = {"latitude": 45.4408,"longitude": 12.3155}  # Venice, Italy

# Get the shortest path between Shanghai, China and Xiamen, China using Marnet, world highways, and world railways geographs
origin = {"latitude": 31.2304, "longitude": 121.4737}  # Shanghai, China
destination = {"latitude": 24.4798, "longitude": 118.0895}  # Xiamen, China

# Get the shortest path between Sydney, Austrailia and Perth, Austrailia using Marnet, world highways, and world railways geographs
# origin = {"latitude": -33.8688,"longitude": 151.2093}  # Sydney, Austrailia
# destination = {"latitude": -31.9505,"longitude": 115.8605}  # Perth, Austrailia

# Get the shortest path between Rio de Janeiro, Brazil and São Paulo, Brazil using Marnet, world highways, and world railways geographs
# origin = {"latitude": -22.9068,"longitude": -43.1729}  # Rio de Janeiro, Brazil
# destination = {"latitude": -23.5505,"longitude": -46.6333}  # São Paulo, Brazil


# Get the shortest path between New York City, USA and Los Angeles, USA using Marnet, world highways, and world railways geographs
marnet_output = marnet_geograph.get_shortest_path(
    origin_node=origin,
    destination_node=destination,
    node_addition_type="quadrant",
    node_addition_lat_lon_bound=3,
    destination_node_addition_type="all"
)

world_highways_output = world_highways_geograph.get_shortest_path(
    origin_node=origin,
    destination_node=destination,
    node_addition_type="quadrant",
    node_addition_lat_lon_bound=3,
    destination_node_addition_type="all"
)

world_railways_output = world_railways_geograph.get_shortest_path(
    origin_node=origin,
    destination_node=destination,
    node_addition_type="quadrant",
    node_addition_lat_lon_bound=3,
    destination_node_addition_type="all"
)


In [ ]:
# A simple example of how to use folium to visualize the shortest path between two points
import folium

# Folium does not wrap coordinates around the globe, so we need to adjust the path accordingly
# Essentially we create 5 differnt paths that offset the original path by 0, 360, -360, 720, -720 degrees
# This allows the illusion that the path wraps around the globe with folium
def adjustArcPath(path):
    for index in range(1, len(path)):
        x = path[index][1]
        prevX = path[index - 1][1]
        path[index][1] = x - (round((x - prevX)/360,0) * 360)
    return path

def modifyArcPathLong(points, amount):
    return [[i[0], i[1]+amount] for i in points]

def getCleanArcPath(path):
    path = adjustArcPath(path)
    return [
        path,
        modifyArcPathLong(path, 360),
        modifyArcPathLong(path, -360),
        modifyArcPathLong(path, 720),
        modifyArcPathLong(path, -720)
    ]

# Create a folium map
map = folium.Map([31.2304, 121.4737], zoom_start=6)
# Populate it with the path
folium.PolyLine(
    getCleanArcPath(marnet_output['coordinate_path']),
    color='green',
    weight=5,
    opacity=0.5,
    popup='Length (KM): ' + str(marnet_output['length'])
).add_to(map)

folium.PolyLine(
    getCleanArcPath(world_highways_output['coordinate_path']),
    color='blue',
    weight=5,
    opacity=0.5,
    popup='Length (KM): ' + str(world_highways_output['length'])
).add_to(map)

folium.PolyLine(
    getCleanArcPath(world_railways_output['coordinate_path']),
    color='red',
    weight=5,
    opacity=0.5,
    popup='Length (KM): ' + str(world_railways_output['length'])
).add_to(map)

# Display a Legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px;
                 padding: 10px;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">
         <h3>Legend<h3/>
         <h4>
         <i class="fa fa-circle" style="color:green"></i>&nbsp;Maritime Network<br>
         <i class="fa fa-circle" style="color:blue"></i>&nbsp;Highway Network<br>
         <i class="fa fa-circle" style="color:red"></i>&nbsp;Railway Network
         </h4>
      </div>
     '''
map.get_root().html.add_child(folium.Element(legend_html))
map